# Исследование надежности заемщиков

###  План работы:
* [Введение](#introduction)
* [Обзор данных](#overview)
* [Предобработка данных](#preprocessing)
* [Проверка гипотез](#hypothesis)
* [Вывод](#conclusion)

<a id="introduction"></a>
## Введение
###  Описание проекта
* Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
* Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

####  Описание данных
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

<a id="overview"></a>
## Обзор данных

In [1]:
import pandas as pd

try:
    df = pd.read_csv('C:/Users/Danila30/Downloads/GitHub/Исследование надежности заемщиков/data.csv', delimiter=',') 
except:
    df = pd.read_csv('/datasets/data.csv', delimiter=',')

In [2]:
display(df.head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<a id="preprocessing"></a>
## Предобработка данных

### Шаг 2.1 Заполнение пропусков

Посмотрим общую информацию всего датасета

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Узнаем количство пропусков в процентах столбацах total_income и days_employed  

In [4]:
share = 19351/21525*100
print(share)

89.90011614401858


Пропущенные значение 'NaN' наблюдаются в столбцах days_employed и total_income, они содержат в себе количественные значения. 
Скорее всего эти пропуски возникли из-за человеческого фактора, данные просто не были внесены. 
Пропуски в таких переменных заполняют характерными значениями. Это значения, характеризующие состояние выборки, — набора данных, выбранных для проведения исследования. Чтобы примерно оценить типичные значения выборки, в данной ситуации лучше использовать медиану!

In [5]:
df[df['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Вычислим медианные значения в столбцах с пропускам, затем заменим пропуски медианным значением.

In [6]:
days_employed_median = df['days_employed'].median()
df['days_employed'] = df['days_employed'].fillna(value = days_employed_median) 

In [7]:
total_income_median = df['total_income'].median()
df['total_income'] = df['total_income'].fillna(value = total_income_median)

Выполним проверку  пропущенных значений по всей таблице

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


С помощью median() удалось найти медиану по заданому столбцу, а метод fillna() понадобился для того, что бы заменить все значения NaN в данных столбцах на медиану.

### Шаг 2.2 Проверка данных на аномалии

В столбце days_employed убираем дробную часть и избавляемся от отрицательных зачений с помощью метода abs(

In [9]:
df['days_employed'] = df['days_employed'].astype('int') 
df['days_employed'] = df['days_employed'].abs()
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Шаг 2.3. Изменение типов данных.

Приведём вещественные значения из столбца total_income в целочисленные. Для этого необходимо применить метод astype(), указав в качестве аргумента int!

In [10]:
df['total_income'] = df['total_income'].astype('int') 
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


#### Оптимизация хранения числовых данных с использованием подтипов

Мы будем часто выяснять то, сколько памяти используется, поэтому создадим функцию, которая поможет нам сэкономить немного времени.

In [11]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # исходим из предположения о том, что если это не DataFrame, то это Series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # преобразуем байты в мегабайты
    return "{:03.2f} MB".format(usage_mb)

Функцию pd.to_numeric() можно использовать для нисходящего преобразования числовых типов. Для выбора целочисленных столбцов воспользуемся методом DataFrame.select_dtypes(), затем оптимизируем их и сравним использование памяти до и после оптимизации.

In [12]:
df_int = df.select_dtypes(include=['int'])
converted_int = df_int.apply(pd.to_numeric,downcast='unsigned')

Вот что получается в результате исследования потребления памяти:

In [13]:
print(mem_usage(df_int))
print(mem_usage(converted_int))

0.99 MB
0.41 MB


В результате можно видеть падение использования памяти с 1,15 до 0,41 мегабайт, то есть — мы снизили потребление памяти больше, чем на 65%.

#### Оптимизация хранения данных объектных типов с использованием категориальных переменных

Для того чтобы понять, где именно мы сможем воспользоваться категориальными данными для снижения потребления памяти, выясним количество уникальных значений в столбцах, хранящих значения объектных типов:

In [14]:
df_obj = df.select_dtypes(include=['object']).copy()
df_obj.describe()

,education,family_status,gender,income_type,purpose
count,21525,21525,21525,21525,21525
unique,15,5,3,8,38
top,среднее,женат / замужем,F,сотрудник,свадьба
freq,13750,12380,14236,11119,797


Прежде чем заняться полномасштабной оптимизацией необходимо выбрать какой-нибудь один столбец, хранящий объектные данные, например income_type, необходимо посмотреть, что происходит внутри программы при преобразовании его в категориальный тип.

В этом столбце содержится всего 8 уникальных значений. Для преобразования его в категориальный тип воспользуемся методом .astype().

In [15]:
dow = df_obj.income_type
print(dow.head())

dow_cat = dow.astype('category')
print(dow_cat.head())

0    сотрудник
1    сотрудник
2    сотрудник
3    сотрудник
4    пенсионер
Name: income_type, dtype: object
0    сотрудник
1    сотрудник
2    сотрудник
3    сотрудник
4    пенсионер
Name: income_type, dtype: category
Categories (8, object): ['безработный', 'в декрете', 'госслужащий', 'компаньон', 'пенсионер', 'предприниматель', 'сотрудник', 'студент']


Хоть тип столбца изменился, данные, хранящиеся в нём, выглядят так же, как и раньше. Посмотрим теперь на то, что происходит внутри программы.

В следующем коде мы используем атрибут Series.cat.codes для того, чтобы выяснить то, какие целочисленные значения тип category использует для представления каждого из типов дохода:

In [16]:
dow_cat.head(9).cat.codes

0    6
1    6
2    6
3    6
4    4
5    3
6    3
7    6
8    6
dtype: int8

Тут можно заметить то, что каждому уникальному значению назначено целочисленное значение, и то, что столбец теперь имеет тип int8. Здесь нет отсутствующих значений, но если бы это было так, для указания таких значений использовалось бы число -1.

Теперь давайте сравним потребление памяти до и после преобразования столбца income_type к типу category.

In [17]:
print(mem_usage(dow))
print(mem_usage(dow_cat))

2.13 MB
0.02 MB


Как видно, сначала потреблялось 2.45 мегабайт памяти, а после оптимизации — лишь 0.02 мегабайт, что означает 99% улучшение этого показателя. Обратите внимание на то, что работа с этим столбцом, вероятно, демонстрирует один из наиболее выгодных сценариев оптимизации, когда в столбце, содержащем примерно 21525 элементов, используется лишь 8 уникальных значений.

Нам стоит ограничить использование типа category, в основном, столбцами, хранящими данные типа object, в которых уникальными являются менее 50% значений. Если все значения в столбце уникальны, то использование типа category приведёт к повышению уровня использования памяти.

Создадим цикл, который перебирает все столбцы, хранящие данные типа object, выясняет, не превышает ли число уникальных значений в столбцах 50%, и если это так, преобразует их в тип category.

In [18]:
converted_obj = pd.DataFrame()

for col in df_obj.columns:
    num_unique_values = len(df_obj[col].unique())
    num_total_values = len(df_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = df_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = df_obj[col]

In [19]:
print(mem_usage(df_obj))
print(mem_usage(converted_obj))

10.86 MB
0.11 MB


Как видно, объём памяти, необходимый для работы со столбцами, хранящими данные типа object, снизился с 12.34 мегабайт до 0.11 мегабайт, то есть на 99%

### Шаг 2.4. Удаление дубликатов.

In [20]:
df.duplicated().sum()

54

Выявим общее количество явных дубликатов. После этого удалим явные дубликаты методом drop_duplicates().reset_index(), с параметром (drop = True), чтобы перезаписать индексы датасета и удалить старые. Затем снова проверим на количество явных дубликатов.

In [21]:
df = df.drop_duplicates().reset_index(drop = True)
df.duplicated().sum()

0

В столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведём их к одному регистру с помощью функции str.lower(), после чего выполним проверку на неявные дубликаты в данном столбце.

In [22]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [23]:
df['education'] = df['education'].str.lower() 
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [24]:
df.duplicated().sum()

17

In [25]:
df = df.drop_duplicates().reset_index(drop = True)
df.duplicated().sum()

0

Данные дубликаты вполне могли появиться из-за человеского фактора(заполнения данных вручную)

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [26]:
df_family = pd.DataFrame(data = df, columns=['family_status_id', 'family_status']) # создаем новый датафрейм с колонками  
# family_status_id и family_status 
df_family = df_family.drop_duplicates().reset_index(drop=True) # удаляем дубликаты в таблице df_family и сбрасываем индексы 
print('Количество дубликатов:', df_family.duplicated().sum()) # проверяем результат отсутствия дубликатов 
display(df_family)
df_education = pd.DataFrame(data = df, columns=['education_id', 'education']) # создаем новый датафрейм с колонками  
# education_id и education
df_education = df_education.drop_duplicates().reset_index(drop=True) # удаляем дубликаты в таблице df_education и сбрасываем индексы 
print('Количество дубликатов:', df_education.duplicated().sum()) # проверяем результат отсутствия дубликатов 
display(df_education)

df = df.drop(['education', 'family_status'], axis =1) # удаляем из исходного датафрейма колонки education и family_status 
display(df.head()) # проверяем результат выводом 5-ти строк таблицы 

Количество дубликатов: 0


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Количество дубликатов: 0


,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

In [27]:
def income_category(category): # создаём функцию-фильтр, который определяет категорию дохода согласно установленым значениям 
    if category <= 30000:
        return 'E'
    elif category >= 30001 and category <= 50000:
        return 'D'
    elif category >= 50001 and category <= 200000:
        return 'C'
    elif category >= 200001 and category <= 1000000:
        return 'B'
    elif category >= 1000001:
        return 'A'

In [28]:
df['total_income_category'] = df['total_income'].apply(income_category) 
display(df.head())
# создаём новый столбец total_income_category в котором содержаться отфильтрованные категории доходов 

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

In [29]:
df['purpose'].unique() # выявляем уникальные значения столбца purpose, для определения значений функции-фильтра

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [30]:
def purpose_category(purpose): # создаём функцию фильтр, для категоризации целей кредита
    if 'авто' in purpose:
        return 'операции с автомобилем'
    elif 'жил' in purpose or 'недвиж' in purpose:
        return 'операции с недвижимостью'
    elif 'свад' in purpose:
        return 'проведение свадьбы'
    elif 'обра' in purpose:
        return 'получение образования'

In [31]:
df['purpose_category'] = df['purpose'].apply(purpose_category)
display(df.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


<a id="hypothesis"></a>
## Проверка гипотез

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Применим категоризацию для заёмщиков по количетсву детей и перезапишем стобец. Т.к. заёмщиков с количеством детей >2 совсем мало!

In [32]:
def count_children(count): 
    if count == 0:
        return '0'
    elif count == 1:
        return '1'
    elif count == 2:
        return '2'
    elif count == 3 and count <= 5:
        return 'более 2-х'

In [33]:
df['children'] = df['children'].apply(count_children) 
display(df.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,более 2-х,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


In [34]:
pivot_children = df.pivot_table(index = 'children', values = 'debt', aggfunc = ['sum', 'count']) 
pivot_children.reset_index()
pivot_children ['debt %'] = pivot_children ['sum'] / pivot_children ['count'] * 100
#создаём сводную таблицу, в которой сумируем количество невозвращённых кредитов по количеству детей и считаем общее количество кредитов, 
#удаляем мультииндексы, проверяем зависимость между количеством детей и возвратом кредита в срок
display(pivot_children)

,sum,count,debt %
,debt,debt,
children,,,
0,1063,14091,7.543822
1,444,4808,9.234609
2,194,2052,9.454191
более 2-х,27,330,8.181818


##### Вывод 1: 

Больше всего кредитов у клиентов с 0 детьми, но при этом процент просроченных кредитов по данной категории самый низкий(7,53%). Больше всего просроченных кредитов наблюдается в категориис 1 ребёнком (444 кредита), а процент невозврата 9,23% (3 место).Заёмщики с более 2-мя детьми имеют составляют очень маленькую долю! Основная целевая аудитория кредитования, клиенты у которых в семье 0,1 и 2 ребёнка.

#####  Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [35]:
df_pivot_family = df.pivot_table(index = 'family_status_id', values = 'debt', aggfunc = ['sum', 'count'])#строим сводную таблицу:  
# в качестве индекса строк выбираем колонку family_status_id (семейный статус),  
df_pivot_family = df_pivot_family.reset_index() # убираем мультииндексы
df_pivot_family.columns = ['family_status_id','sum_debt','count_debt'] # изменяем название колонок
# в колонке sum debt считаем сумму задолжности, к колонке count debt считаем количество должников 
df_pivot_family['debt %'] = df_pivot_family['sum_debt'] / df_pivot_family['count_debt'] * 100 # добавляем в таблицу колонку  debt %  
# с расчетом процентного соотношения суммы долга к количеству должников 

df_pivot_family_merged = df_pivot_family.merge(df_family, on='family_status_id', how = 'left') # объединяем таблицу со "словарем"  
# df_family по общей колонке family_status_id  
 
display(df_pivot_family_merged) # выводим объединенную таблицу на экран 
 

,family_status_id,sum_debt,count_debt,debt %,family_status
0,0,931,12339,7.545182,женат / замужем
1,1,388,4151,9.347145,гражданский брак
2,2,63,959,6.569343,вдовец / вдова
3,3,85,1195,7.112971,в разводе
4,4,274,2810,9.750890,Не женат / не замужем


##### Вывод 2: 

Больше всего количетсво взятых и просроченных кредитов в категории 0, но по проценту просроченности данная категория занимает 3 место (7,54%). 4 категория по общему количеству кредитов занимает 3 место, но при этом, чаще всего просрочивает возврат кредита в банк (9,75%). Вдовы и вдовцы реже всего задерживают выплату кредита в срок. Клиенты состоящие в гражданском браке занимают 2 место по общему количеству кредитов и по задержке в его выплате!

#####  Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [36]:
df_pivot_income = df.pivot_table(index=['total_income_category'], columns='debt', values='children', aggfunc='count') 
df_pivot_income['ratio %'] = df_pivot_income[1] / (df_pivot_income[0] + df_pivot_income[1])*100 
#print(f'Группа A: {df_pivot_income.loc[1, "ratio%"]:.1%}')
display(df_pivot_income)


debt,0,1,ratio %
total_income_category,,,
A,23,2,8.000000
B,4649,353,7.057177
C,14534,1350,8.499119
D,327,21,6.034483
E,20,2,9.090909


##### Вывод 3: 

Люди с самым низким с самым высоким доходом берут кредиты намного реже остальных. Основной сегмент получающий кредиты начинается с категории дохода B по D. Чаще всего кредиты берут клиенты со средним доходом (категория C) по выплате кредита не в срок они занимают 2 место (8,48%). Категория B чуть более порядочная занимает 4 место (7,06%). Категория D возвращает кредиты быстрее всех (6,00%).

#####  Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [37]:
df_pivot_purpose = df.pivot_table(index=['purpose_category'], columns='debt', values='children', aggfunc='count') 
df_pivot_purpose['ratio %'] = df_pivot_purpose[1] / (df_pivot_purpose[0] + df_pivot_purpose[1])*100
display(df_pivot_purpose)


debt,0,1,ratio %
purpose_category,,,
операции с автомобилем,3868,399,9.350832
операции с недвижимостью,9948,777,7.244755
получение образования,3611,369,9.271357
проведение свадьбы,2126,183,7.925509


##### Вывод 4: 

Люди берущие кредиты на операции с недвижимостью(7,23%) и проведение свадьбы(7,96%), гораздо реже опаздывают с погашением кредита в срок. Люди берущие кредит на образование(9,21%) и операции с автомобилем (9,35%).

<a id="conclusion"></a>
## Вывод:

### Основная целевая аудитория кредитования
* Клиенты у которых в семье 0 детей - 14091 кредитов из них 1063 не погасили в срок (7.54%).
* Супружеские пары, состоящие в браке - 12339 кредитов из них 931 не погашены (7.54%).
* Люди со средним доходом 50001 - 200000 т.р.(категория C) - 14534 кредитов из них 1350 вовремя не погасили	(8.49%). 
* Клиенты, которые берут кредит на операции с недвижимостью - 9948 кредитов из них 777 не погасили в срок (7.24%).

Т.е. в основном это молодые люди которые женятся или только поженились или люди средних лет у которых средний доход и нету детей.

### Клиенты с худшим соотношением выплаты кредитов в срок
* Клиенты у которых в семье 2 ребёнка - 194 задолжности из 2052 (9.45%).
* Люди которые не состоят в браке - 274	просроченых кредита из 2810	(9.75%).
* Люди со средним доходом 50001 - 200000 т.р.(категория C), т.к. категория Е содержит очень мало клиентов - тут 14534 клиента 1350 из них не успели вовремя выплатить кредит (8.49%).
* Клиенты, которые берут кредит на операции с автомобилем - всего их 3868 из которых 399 вовремя не выплатили кредит (9.35%).

В итоге стало понятно, хуже всег овыплачивают кредит люди со средним доходом, т.к. на них приходится больше всего выданных и непогашенных кредитов!

Другие самые объёмные категории не имеют самых низкиз показателей по погашению кредита в срок!